In [1]:
import bs4
import json
import requests
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [2]:
es.indices.delete(index='my_corp', ignore=[400, 404])

{u'acknowledged': True}

In [54]:
def getTrans(link):
    page = requests.get(link)
    soup = bs4.BeautifulSoup(page.text)
    return soup.find('div',{'id':'transcript'}).findAll(text = True)
    

In [55]:
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

In [56]:
def createJSON (link, text):
    lnk = 'http://millercenter.org' + link
    pres_id = link[findnth(link,'/',1)+1 : findnth(link,'/',2)]
    speech_id = link[findnth(link,'/',3)+1 : len(link)]
    ps = {}
    ps['transcript'] = '' #getTrans(lnk)
    ps['link'] = lnk
    ps['title'] = text[0:text.find("(")]
    ps['pres_id'] = pres_id
    ps['es_id'] = pres_id +'_'+ speech_id
    ps['date'] = text[text.find("(")+1:text.find(")")]
    return ps


In [57]:
def writeFile (data):
    with open('data.txt', 'a') as outfile:
        json.dump(data, outfile)

In [59]:
page = requests.get('http://millercenter.org/president/speeches')
#print page.text
soup = bs4.BeautifulSoup(page.text)
#links = soup.select('div.entry a[href^=/president]')
links = soup.select('div.title a[href^=/president]')
#print len(links)
for link in links:
    print link.attrs['href']
    ps = createJSON(link.attrs['href'],link.text)
    #writeFile(ps)  #test first before overwriting this file

In [66]:
import json

with open('data.txt') as data_file:    
    data = json.load(data_file)

for link in data['data']:
    link['transcript'] = getTrans(link['link'])
    es.index(index='presidential', doc_type='speeches', id=link['es_id'], body = link)
    
print 'done'

done
